# Matrix-free preconditioning

We combine a matrix-free operator with a matrix-free preconditioner. This allows to solve huge high-order problems with little memory

In [1]:
import numpy as np
import ngsolve.comp
from ngsolve import *
from ngsolve.webgui import Draw

from ngsolve.krylovspace import CGSolver
from time import time

In [2]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))

In [ ]:
order = 20

fes = H1(mesh, order=order, dirichlet='left|bottom', wb_withedges=False)
print ("ndof =", fes.ndof)
fes2 = VectorL2(mesh, order=order-1, covariant=True)
fesl2 = L2(mesh, order=1)

u,v = fes.TnT()

from ngsolve.comp import ConvertOperator
opgrad = ConvertOperator(fes, fes2, grad(u), geom_free=True)
opid = ConvertOperator(fes, fesl2, geom_free=True)

feslo = H1(mesh, order=1, dirichlet='left|bottom')
ulo,vlo = feslo.TnT()
alo = BilinearForm(grad(ulo)*grad(vlo)*dx).Assemble()
loinv = alo.mat.Inverse(feslo.FreeDofs())
prelo = fes.loembedding@loinv@fes.loembedding.T

In [ ]:
Laplace = opgrad.T @ fes2.Mass(1) @ opgrad 
Mass = opid.T @fesl2.Mass(1) @ opid

K = Laplace+Mass
pre = IdentityMatrix(fes.ndof)

In [ ]:
bfi = (grad(u)*grad(v)*dx)[0].MakeBFI()
def CreatorLocal(ei):
    # print (ei)
    mat = bfi.CalcElementMatrix(fes.GetFE(ei), mesh.GetTrafo(ei))
    dofs = fes.GetDofNrs(ei)
    # print (dofs)
    local = [i for (i,d) in enumerate(dofs) if fes.CouplingType(d)==COUPLING_TYPE.LOCAL_DOF]
    # print (local)
    matlocal = Matrix( np.asmatrix(mat)[local,:][:,local]).I
    # print (matlocal)
    return tuple([matlocal, local, local])
prelocal = ngsolve.comp.MatrixFreeOperator(fes, CreatorLocal)

def CreatorExt(ei):
    mat = bfi.CalcElementMatrix(fes.GetFE(ei), mesh.GetTrafo(ei))
    dofs = fes.GetDofNrs(ei)    
    local = [i for (i,d) in enumerate(dofs) if fes.CouplingType(d)==COUPLING_TYPE.LOCAL_DOF]
    coupling = [i for (i,d) in enumerate(dofs) if fes.CouplingType(d)!=COUPLING_TYPE.LOCAL_DOF]
    matLL = Matrix( np.asmatrix(mat)[local,:][:,local])
    matLC = Matrix( np.asmatrix(mat)[local,:][:,coupling])
    mat = -matLL.I*matLC
    return tuple([mat, coupling, local])
ext = IdentityMatrix(fes.ndof)+ngsolve.comp.MatrixFreeOperator(fes, CreatorExt)

def CreatorInterface(ei):
    mat = bfi.CalcElementMatrix(fes.GetFE(ei), mesh.GetTrafo(ei))
    dofs = fes.GetDofNrs(ei)    
    local = [i for (i,d) in enumerate(dofs) if fes.CouplingType(d)==COUPLING_TYPE.LOCAL_DOF]
    interface = [i for (i,d) in enumerate(dofs) if fes.CouplingType(d)==COUPLING_TYPE.INTERFACE_DOF]
    matIF = Matrix( np.asmatrix(mat)[interface,:][:,interface]).I
    return tuple([matIF, interface, interface])
preinterface = ngsolve.comp.MatrixFreeOperator(fes, CreatorInterface)


coupling = fes.FreeDofs(True)
proj = Projector(coupling, True)

# pre = prelocal + proj
pre = prelocal + ext@(proj@preinterface@proj + prelo)@ext.T 
# pre = prelocal + proj@preinterface@proj

print (pre.GetOperatorInfo())

In [ ]:
f = LinearForm(x*v*dx).Assemble()
gfu = GridFunction(fes)

In [ ]:
ts = time()
with TaskManager():
    inv = CGSolver(K, pre, printrates=True, maxiter=500)
    gfu.vec.data = inv * f.vec
print ("Solved ", fes.ndof, " in ", time()-ts)

In [ ]:
Draw (gfu, order=8);